In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

np.random.seed(20160704)
tf.set_random_seed(20160704)

mnist data를 불러옴

In [0]:

mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


첫 번째 단계의 합성곱필터와 폴링계층을 정의

In [0]:
num_filters1 = 32
num_filters2 = 64

num_units1 = 7*7*num_filters2
num_units2 = 1024

x = tf.placeholder(tf.float32, [None, 784])
x_image = tf.reshape(x, [-1,28,28,1])

W_conv1 = tf.get_variable("w_conv1",[5,5,1,num_filters1],initializer=tf.contrib.layers.xavier_initializer())
h_conv1 = tf.nn.conv2d(x_image, W_conv1,strides=[1,1,1,1], padding='SAME')

b_conv1 = tf.Variable(tf.constant(0.1, shape=[num_filters1]))
h_conv1_cutoff = tf.nn.relu(h_conv1 + b_conv1)

h_pool1 = tf.nn.max_pool(h_conv1_cutoff, ksize=[1,2,2,1],strides=[1,2,2,1], padding='SAME')

두 번째 단계의 합성곱필터와 폴링계층을 정의

In [0]:

W_conv2 = tf.get_variable("w_conv2",[5,5,num_filters1, num_filters2],initializer=tf.contrib.layers.xavier_initializer())

h_conv2 = tf.nn.conv2d(h_pool1, W_conv2,strides=[1,1,1,1], padding='SAME')

b_conv2 = tf.Variable(tf.constant(0.1, shape=[num_filters2]))
h_conv2_cutoff = tf.nn.relu(h_conv2 + b_conv2)

h_pool2 = tf.nn.max_pool(h_conv2_cutoff, ksize=[1,2,2,1],strides=[1,2,2,1], padding='SAME')

 결합층, 드롭아웃 계층, 소프트맥스 함수를 정의한다.

In [0]:
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*num_filters2])


w1 = tf.get_variable("w1",[num_units1,num_units2],initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.constant(0.1, shape=[num_units2]))
hidden2 = tf.nn.relu(tf.matmul(h_pool2_flat, w1) + b1)

keep_prob = tf.placeholder(tf.float32)
hidden2_drop = tf.nn.dropout(hidden2, keep_prob)

w2 = tf.get_variable("w2",[num_units2,10],initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.zeros([10]))
p = tf.nn.softmax(tf.matmul(hidden2_drop, w2) + b2)

loss, optimizer, accuracy을 정의한다.

In [0]:
t = tf.placeholder(tf.float32, [None, 10])
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=p, labels=t))
optimizer = tf.train.AdamOptimizer(0.0001).minimize(loss)
correct_prediction = tf.equal(tf.argmax(p, 1), tf.argmax(t, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

세션을 준비한다

In [0]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

학습을 20000회반복한다. float의 문제로 nan이 나오는 것 같다.

In [0]:
i = 0
for _ in range(20000):
    i += 1
    batch_xs, batch_ts = mnist.train.next_batch(100)
    sess.run(optimizer,feed_dict={x:batch_xs, t:batch_ts, keep_prob:0.5})
    if i % 1000 == 0:
        loss_val, acc_val = sess.run([loss, accuracy],feed_dict={x:mnist.test.images, t:mnist.test.labels,keep_prob:1.0})
        print ('Step: %d, Loss: %f, Accuracy: %f'% (i, loss_val, acc_val))

Step: 1000, Loss: 838.974243, Accuracy: 0.975100
Step: 2000, Loss: 452.666809, Accuracy: 0.985500
Step: 3000, Loss: 337.046936, Accuracy: 0.988700
Step: 4000, Loss: 290.856262, Accuracy: 0.990100
Step: 5000, Loss: 268.446045, Accuracy: 0.990800
Step: 6000, Loss: 248.016510, Accuracy: 0.991900
Step: 7000, Loss: 223.197495, Accuracy: 0.992400
Step: 8000, Loss: 217.023804, Accuracy: 0.992600
Step: 9000, Loss: 225.896973, Accuracy: 0.992200
Step: 10000, Loss: 207.161224, Accuracy: 0.992600
Step: 11000, Loss: 244.336517, Accuracy: 0.992000
Step: 12000, Loss: 253.727264, Accuracy: 0.993100
Step: 13000, Loss: 224.667465, Accuracy: 0.992900
Step: 14000, Loss: 212.106415, Accuracy: 0.993500
Step: 15000, Loss: 240.525787, Accuracy: 0.993000
Step: 16000, Loss: 240.936127, Accuracy: 0.992300
Step: 17000, Loss: 235.511627, Accuracy: 0.992800
Step: 18000, Loss: 267.905029, Accuracy: 0.992000
Step: 19000, Loss: nan, Accuracy: 0.098000
Step: 20000, Loss: nan, Accuracy: 0.098000
